# Install dependencies : Tensorflow, keras, keras-RL , open-AI Gym

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

  Using cached tensorflow-2.3.0-cp37-cp37m-win_amd64.whl (342.5 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Rolling back uninstall of tensorflow
  Moving to c:\users\leilaa\appdata\roaming\python\python37\scri

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\leilaa\\anaconda3\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\leilaa\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leilaa\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leilaa\anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached gast-0.2.2-py3-none-any.whl
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


You should consider upgrading via the 'c:\users\leilaa\anaconda3\python.exe -m pip install --upgrade pip' command.


# Test random environment with OpenAI Gym

In [3]:
import gym
import random

In [11]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]

In [6]:
states

4

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    env.reset()
    score = 0
    done = False
    
    while not done:
        env.render()
        action = random.choice([0,1])
        state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{}, Score:{}'.format(episode, score))
env.close()

Episode:1, Score:18.0
Episode:2, Score:32.0
Episode:3, Score:22.0
Episode:4, Score:14.0
Episode:5, Score:11.0
Episode:6, Score:15.0
Episode:7, Score:10.0
Episode:8, Score:15.0
Episode:9, Score:21.0
Episode:10, Score:22.0


# Create a deep learning model with Keras

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))    
    return model

In [12]:
model = build_model(states, actions)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build agent with Keras-RL

In [30]:
from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [6]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    sarsa = SARSAAgent(model, actions, policy=None, test_policy=None, gamma=0.99, nb_steps_warmup=10, train_interval=1)

#     dqn = DQNAgent(model=model, memory=memory, policy=policy
#                   ,nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return sarsa

In [37]:
sarsa = build_agent(model, actions)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])
sarsa.fit(env, nb_steps=2000, visualize=False)

Training for 2000 steps ...
Interval 1 (0 steps performed)
 1998/10000 [====>.........................] - ETA: 2:28 - reward: 1.0000- ETA: 2:30 - rewarddone, took 37.164 seconds


In [39]:
score = sarsa.test(env, nb_episodes=100, visualize=False)
print(np.mean(score.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 21.000, steps: 21
Episode 2: reward: 21.000, steps: 21
Episode 3: reward: 21.000, steps: 21
Episode 4: reward: 24.000, steps: 24
Episode 5: reward: 26.000, steps: 26
Episode 6: reward: 25.000, steps: 25
Episode 7: reward: 29.000, steps: 29
Episode 8: reward: 23.000, steps: 23
Episode 9: reward: 22.000, steps: 22
Episode 10: reward: 32.000, steps: 32
Episode 11: reward: 23.000, steps: 23
Episode 12: reward: 21.000, steps: 21
Episode 13: reward: 26.000, steps: 26
Episode 14: reward: 17.000, steps: 17
Episode 15: reward: 19.000, steps: 19
Episode 16: reward: 20.000, steps: 20
Episode 17: reward: 23.000, steps: 23
Episode 18: reward: 26.000, steps: 26
Episode 19: reward: 25.000, steps: 25
Episode 20: reward: 19.000, steps: 19
Episode 21: reward: 23.000, steps: 23
Episode 22: reward: 18.000, steps: 18
Episode 23: reward: 29.000, steps: 29
Episode 24: reward: 19.000, steps: 19
Episode 25: reward: 26.000, steps: 26
Episode 26: reward: 25.000, st

In [41]:
_ = sarsa.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 28.000, steps: 28
Episode 2: reward: 25.000, steps: 25
Episode 3: reward: 24.000, steps: 24
Episode 4: reward: 23.000, steps: 23
Episode 5: reward: 21.000, steps: 21
Episode 6: reward: 21.000, steps: 21
Episode 7: reward: 21.000, steps: 21
Episode 8: reward: 17.000, steps: 17
Episode 9: reward: 25.000, steps: 25
Episode 10: reward: 32.000, steps: 32
Episode 11: reward: 20.000, steps: 20
Episode 12: reward: 23.000, steps: 23
Episode 13: reward: 21.000, steps: 21
Episode 14: reward: 21.000, steps: 21
Episode 15: reward: 20.000, steps: 20


# Reloading agent from memory

In [42]:
sarsa.save_weights('sarsa.weights.h5f', overwrite=True)

In [43]:
del sarsa
del env
del model

In [7]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
sarsa = build_agent(model, actions)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


NameError: name 'BoltzmannQPolicy' is not defined

In [49]:
sarsa.load_weights('sarsa.weights.h5f')

In [1]:
sarsa.test(env, nb_episodes=5, visualize=True)

NameError: name 'sarsa' is not defined

In [53]:
env.close()